In [36]:
import os
import numpy as np
import time
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
seed = 5693
np.random.seed(seed)
tf.random.set_seed(seed)
NUM_OF_CATEGORY = 10

In [37]:
# mnist = tf.keras.datasets.fashion_mnist
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_final_test, y_final_test = x_test, y_test

x_test = x_train[50000:60000]
x_train = x_train[0:50000]
y_test = y_train[50000:60000]
y_train = y_train[0:50000]

x_train, x_test, x_final_test = np.reshape(x_train / 255.0, (int(tf.shape(x_train)[0]),-1)), np.reshape(x_test / 255.0, (int(tf.shape(x_test)[0]),-1)), np.reshape(x_final_test / 255.0, (int(tf.shape(x_final_test)[0]),-1))
one_hot_y = lambda t: [1 if i == t else 0 for i in range(NUM_OF_CATEGORY)]
y_train, y_test, y_final_test = np.array([one_hot_y(y) for y in y_train]), np.array([one_hot_y(y) for y in y_test]), np.array([one_hot_y(y) for y in y_final_test])


In [38]:
class MLP(tf.keras.Model):
    def __init__(self, size_input, size_hidden, size_output, device=None):
        super(MLP, self).__init__()
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        """
        self.size_input, self.size_hidden, self.size_output, self.device =\
        size_input, size_hidden, size_output, device
        # Initialize weights between input layer and hidden layer 1
        self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden], seed=seed, stddev=0.1))
        # Initialize biases for hidden layer 1
        self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 1 and hidden layer 2
        self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_hidden], seed=seed, stddev=0.1))
        # Initialize biases for hidden layer 2
        self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 2 and output layer
        self.W3 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output], seed=seed, stddev=0.1))
        # Initialize biases for output layer
        self.b3 = tf.Variable(tf.random.normal([1, self.size_output], seed=seed))

        # Define variables to be updated during backpropagation
        self.MLP_variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
        # self.MLP_variables = [self.W1, self.W3, self.b1, self.b3]

    
    def forward(self, X, dropout_rate=0):
        """
        forward pass
        X: Tensor, inputs
        """
        if self.device is not None:
            with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
                self.y = self.compute_output(X, dropout_rate=dropout_rate)
        else:
            self.y = self.compute_output(X, dropout_rate=dropout_rate)

        return self.y
  
    def loss(self, y_pred, y_true, L1=0, L2=0):
        '''
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        l2_penlty = (tf.nn.l2_loss(self.W1)+tf.nn.l2_loss(self.W2)+tf.nn.l2_loss(self.W3))*L2
        loss_with_l2 = l2_penlty+tf.losses.categorical_crossentropy(y_true_tf, y_pred_tf)
#         print("y_true_tf",y_true_tf,"y_pred_tf",y_pred_tf)
        return loss_with_l2
  
    def backward(self, X_train, y_train, dropout_rate, learning_rate, L1=0, L2=0):
        """
        backward pass
        """
#         optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train,dropout_rate=dropout_rate)
            current_loss = self.loss(predicted, y_train, L2=L2)
        grads = tape.gradient(current_loss, self.MLP_variables)
        
        
        optimizer.apply_gradients(zip(grads, self.MLP_variables))
        return current_loss, predicted
        
        
    def compute_output(self, X, dropout_rate=0):
        """
        Custom method to obtain output tensor during forward pass
        """
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        #Remember to normalize your dataset before moving forward
        # Compute values in hidden layer 1
        what = tf.matmul(X_tf, self.W1) + self.b1
        hhat = tf.nn.relu(what)

        # Compute values in hidden layer 2
        what = tf.matmul(hhat, self.W2) + self.b2
        hhat = tf.nn.relu(what)

        # Compute output
        output = tf.matmul(hhat, self.W3) + self.b3
        #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
        #Second add tf.Softmax(output) and then return this variable
        return tf.nn.softmax(output)
#         return output


In [39]:
result_for_ten = []
for s in range(10):
    print(s,"!!!!!!!!!!!!!!!!!")
    DROPOUT_RATE = 0
    BATCH_SIZE = 200
    HIDDEN_SIZE = 128
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    L2_PENLTY = 0

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    path = f"OPT-Adam_SEED_{s}-LearnRate_{LEARNING_RATE}-L2_{L2_PENLTY}-TIME_{current_time}"

#     Set log summary

    train_log_dir = 'logs/mnist/' + path + '/train'
    test_log_dir = 'logs/mnist/' + path + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)


    size_input = int(tf.shape(x_train)[1])
    size_hidden = HIDDEN_SIZE
    size_output = int(tf.shape(y_train)[1])
    number_of_train_examples = int(tf.shape(x_train)[0])
    number_of_test_examples = int(tf.shape(x_test)[0])


    # print("size_input",size_input)
    # print("size_output",size_output)
    # print("number_of_train_examples",number_of_train_examples)
    # print("number_of_test_examples",number_of_test_examples)


    mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='cpu')
    time_start = time.time()
    epoch = 1
    loss_diff,last_loss = 1,0

    while epoch <= NUM_EPOCHS and abs(loss_diff) > 0.00001:
        loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
        train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BATCH_SIZE+BATCH_SIZE//4, seed=epoch*(seed)).batch(BATCH_SIZE)
        for inputs, outputs in train_ds:
            cur_loss, preds = mlp_on_gpu.backward(inputs, outputs, dropout_rate=DROPOUT_RATE, learning_rate=LEARNING_RATE, L2=L2_PENLTY)
            loss_total_gpu += cur_loss
      # Calculate Accuracy
        train_accuracy, test_accuracy = tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.CategoricalAccuracy()
        train_accuracy.update_state(y_train, mlp_on_gpu.forward(x_train))
        test_accuracy.update_state(y_test, mlp_on_gpu.forward(x_test))
        train_loss = np.sum(loss_total_gpu) / x_train.shape[0]
        test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_test), y_test)) / x_test.shape[0]

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss, step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss, step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        
        
        loss_diff = train_loss - last_loss
        last_loss = train_loss
        print(f'Number of Epoch = {epoch} - Training Cross Entropy:= {np.sum(loss_total_gpu) / x_train.shape[0]} - Training Accuracy:= {train_accuracy.result().numpy()} - Test Accuracy:= {test_accuracy.result().numpy()}')
        time_taken = time.time() - time_start
        print('Time taken (in seconds): {:.2f}'.format(time_taken))
        time_start = time.time()
        epoch += 1

    # record loss and accuracy for final test set:
    final_test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_final_test), y_final_test)) / x_final_test.shape[0]
    final_test_acc = tf.keras.metrics.CategoricalAccuracy()
    final_test_acc.update_state(y_final_test, mlp_on_gpu.forward(x_final_test))
    result_for_ten.append([final_test_loss,final_test_acc.result().numpy()])
#     print(result_for_ten)
    
    

0 !!!!!!!!!!!!!!!!!
Number of Epoch = 1 - Training Cross Entropy:= 0.5322098828125 - Training Accuracy:= 0.9084600806236267 - Test Accuracy:= 0.912600040435791
Time taken (in seconds): 4.90
Number of Epoch = 2 - Training Cross Entropy:= 0.234325 - Training Accuracy:= 0.9405200481414795 - Test Accuracy:= 0.9448000192642212
Time taken (in seconds): 4.76
Number of Epoch = 3 - Training Cross Entropy:= 0.17761802734375 - Training Accuracy:= 0.9552600383758545 - Test Accuracy:= 0.9568000435829163
Time taken (in seconds): 5.32
Number of Epoch = 4 - Training Cross Entropy:= 0.143123759765625 - Training Accuracy:= 0.963140070438385 - Test Accuracy:= 0.9618000388145447
Time taken (in seconds): 5.07
Number of Epoch = 5 - Training Cross Entropy:= 0.12083126953125 - Training Accuracy:= 0.9675800800323486 - Test Accuracy:= 0.964400053024292
Time taken (in seconds): 4.90
Number of Epoch = 6 - Training Cross Entropy:= 0.102825244140625 - Training Accuracy:= 0.9733200669288635 - Test Accuracy:= 0.96800

Number of Epoch = 9 - Training Cross Entropy:= 0.068595224609375 - Training Accuracy:= 0.9793200492858887 - Test Accuracy:= 0.9702000617980957
Time taken (in seconds): 4.82
Number of Epoch = 10 - Training Cross Entropy:= 0.060810146484375 - Training Accuracy:= 0.9833600521087646 - Test Accuracy:= 0.9726000428199768
Time taken (in seconds): 4.79
5 !!!!!!!!!!!!!!!!!
Number of Epoch = 1 - Training Cross Entropy:= 0.563668828125 - Training Accuracy:= 0.9044400453567505 - Test Accuracy:= 0.9150000214576721
Time taken (in seconds): 4.72
Number of Epoch = 2 - Training Cross Entropy:= 0.235524453125 - Training Accuracy:= 0.9419000744819641 - Test Accuracy:= 0.9490000605583191
Time taken (in seconds): 4.82
Number of Epoch = 3 - Training Cross Entropy:= 0.17729828125 - Training Accuracy:= 0.9547000527381897 - Test Accuracy:= 0.957800030708313
Time taken (in seconds): 4.71
Number of Epoch = 4 - Training Cross Entropy:= 0.14217369140625 - Training Accuracy:= 0.9627200365066528 - Test Accuracy:= 0.

Number of Epoch = 7 - Training Cross Entropy:= 0.08547451171875 - Training Accuracy:= 0.9753800630569458 - Test Accuracy:= 0.9679000377655029
Time taken (in seconds): 4.77
Number of Epoch = 8 - Training Cross Entropy:= 0.0759436279296875 - Training Accuracy:= 0.9783000349998474 - Test Accuracy:= 0.968500018119812
Time taken (in seconds): 5.24
Number of Epoch = 9 - Training Cross Entropy:= 0.06696216796875 - Training Accuracy:= 0.9803600907325745 - Test Accuracy:= 0.9682000279426575
Time taken (in seconds): 4.92
Number of Epoch = 10 - Training Cross Entropy:= 0.0596779736328125 - Training Accuracy:= 0.9833400845527649 - Test Accuracy:= 0.9709000587463379
Time taken (in seconds): 4.83


In [40]:
result_for_ten

[[0.09765180053710938, 0.96970004],
 [0.0961176513671875, 0.96830004],
 [0.09532408447265625, 0.97120005],
 [0.09724625244140625, 0.9703],
 [0.0962802978515625, 0.9692001],
 [0.09398307495117188, 0.9710001],
 [0.099090234375, 0.9696],
 [0.09253653564453125, 0.9692001],
 [0.10103370361328125, 0.9689],
 [0.09880753784179687, 0.96880007]]

In [41]:
print(sum(map(lambda x: x[0], result_for_ten))/len(result_for_ten),
      sum(map(lambda x: x[1], result_for_ten))/len(result_for_ten))

0.09680711730957031 0.9696200489997864
